# Diferenčno izražanje reakcij
Primerjali bomo pare reakcij (kontrola, utišanje) na podlagi vzorčenih metabolnih pretokov.

Uporabimo lahko npr. test Kolmogorov-Smirnov, ki ne predpostavlja normalne porazdelitve v vzorcih, statistična značilnost razlik pa je ovrednotena s p-vrednostmi.

Dodatno bomo opazovali kako močno se reakcije med vzorci razlikujejo (fold-changes):

$$FC = \frac{\overline{R_{kd}} - \overline{R_{control}}}{\left|\overline{R_{kd}} + \overline{R_{control}}\right|}$$

In [1]:
import pandas as pd
import numpy as np

from scipy.stats import ks_2samp
#import statsmodels.stats.multitest as multi

import os.path

from helpers import bh

### Osnovne nastavitve

In [2]:
require_biomass = True
folder_samples = "samples"
folder_enrich = "enrichment"

### Branje iz datotek

In [3]:
df_control = pd.read_csv(os.path.join(f'{folder_samples}','samples_control.csv'))
df_kd = pd.read_csv(os.path.join(f'{folder_samples}','samples_kd.csv'))

In [4]:
reactions = sorted(list(set(df_control.columns) | set(df_kd.columns)))
len(reactions) # število reakcij

2282

### Diferenčna aktivnost reakcij

In [5]:
df = pd.DataFrame(columns=['reaction', 'FC', 'p', 'q', 'enrichment', 'changed'])
df['reaction']=reactions

n_samples = df_control.shape[0]

# sprehodimo se čez vse reakcije
for reaction in reactions:
    if reaction in df_control.columns:
        control = df_control[reaction].values
    else:
        # če reakcije ni v kontrolni skupini, ji pripišemo same ničle
        control = np.zeros(n_samples)
        
    if reaction in df_kd.columns:
        kd = df_kd[reaction].values
    else:
        # če reakcije ni v kd skupini, ji pripišemo same ničle
        kd = np.zeros(n_samples)
        
    # iztračunamo sredino za kontrolo in kd
    mean_control = np.mean(control)
    mean_kd = np.mean(kd)
    
    # izračunamo FC - fold change in signifikanco z uporabo 2 sample Kolmogorov-Smirnov testa
    if mean_control != 0 or mean_kd != 0:
        FC = (mean_kd-mean_control)/(abs(mean_kd + mean_control))
        p = ks_2samp(control,kd)[1]
    else:
        FC = 0
        p = 1     
        
    df.loc[df['reaction']==reaction, 'FC'] = FC
    df.loc[df['reaction']==reaction, 'p'] = p
    
    
# korigiramo p vrednosti za večkratno testiranje - FDR korekcija
df['q'] = bh(df['p'])

# signifikanca zahteva vsaj 10-kratno up-/down-regulacijo
df.loc[(df['FC'] >= 0.82) & (df['q'] < 0.05),'enrichment'] = 1
df.loc[(df['FC'] <= -0.82) & (df['q'] < 0.05),'enrichment'] = -1
df.loc[~df['enrichment'].isna(),'changed'] = 1
df = df.fillna(0)
    
    

In [10]:
df.to_csv(os.path.join(f"{folder_enrich}","reactions.csv"), index=False)

In [11]:
df[df.enrichment == -1]

,reaction,FC,p,q,enrichment,changed
2,10FTHFtl,-1.0,0.0,0.0,-1,1
7,24_25DHVITD2t,-1.0,0.0,0.0,-1,1
8,24_25DHVITD2tm,-1.0,0.0,0.0,-1,1
9,24_25DHVITD3t,-1.0,0.0,0.0,-1,1
10,24_25DHVITD3tm,-1.0,0.0,0.0,-1,1
...,...,...,...,...,...,...
2269,r2472,-1.0,0.0,0.0,-1,1
2270,r2509,-1.0,0.0,0.0,-1,1
2271,r2510,-1.0,0.0,0.0,-1,1
2272,r2511,-1.0,0.0,0.0,-1,1
